In [5]:
import selenium
from Other.Scraping import *

In [2]:
driver.get("http://www.randhawa.us/games/retailer/nyu.html")

# Define Strategy

In [3]:

import pulp as plp
import numpy as np
def optimizer(a):
    model = plp.LpProblem("Retail_Game", plp.LpMaximize)
    I = range(4)
    X = plp.LpVariable.dicts(name='X', indexs=(I),lowBound=0, cat=plp.LpInteger)
    discounted_price = [60,54,48,36]
    demand_lift = [1,1.31,1.73,2.81]
    model += plp.lpSum(a*discounted_price[i]*demand_lift[i]*X[i] for i in I)
    model += X[0] >= 1
    model += plp.lpSum(a*demand_lift[i]*X[i] for i in I) <= 2000
    model += plp.lpSum(X[i] for i in I) <= 15
    model.solve()
    print("="*30,"\nSolution Status:", plp.LpStatus[model.status])

    # Results
    obj = plp.value(model.objective)
    print("The result is: ${}".format(obj))
    values=[]
    print("Optimal Soultion:")
    for v in model.variables():
        print(v.name, "=", v.varValue)
        values.append(v.varValue)
    return(model,values)
model,choices=optimizer(100)

Solution Status: Optimal
The result is: $106578.0
Optimal Soultion:
X_0 = 3.0
X_1 = 9.0
X_2 = 3.0
X_3 = 0.0


In [6]:
def click_strategy(chosenchoices,week):
    if(week>0):
        if(chosenchoices[week]==chosenchoices[week-1]):
            return(0)
        else:
            return(chosenchoices[week])
    return(chosenchoices[week])
# Get data whenever
# def richard_strategy(table,choices,chosenchoices):
    
#     week=max(table['Week'].astype(int))
#     if(week > 1):
#         prev_choice=chosenchoices[week-1]
#         lift=(int(table['Sales'][week-1])-int(table['Sales'][week-2]))/int(table['Sales'][week-2])
#         lastchoice=max(chosenchoices)
#         remainingweeks=15-week
#         canfinish=False
#         remaining=table['Remaining'][week-1]
#          if(remainingweeks*table['Sales'][week-1]>remaining):
#              canfinish=True
        
#         if(lift<0):
#             print("negative lift: "+ str(lift))
#             return(min(lastchoice+1,3))
#     return(0)



### Get Buttons

In [7]:
html_source = driver.page_source

# Controls


def main_game_click(click_type=0):
#     global click_count
#     if click_count > 15:
#         print('Error: cannot go over 15 weeks.')
#         return
    maintain = driver.find_element_by_id('maintainButton')
    ten = driver.find_element_by_id('tenButton')
    twenty = driver.find_element_by_id('twentyButton')
    forty = driver.find_element_by_id('fortyButton')

    if click_type == 0:
        maintain.click()
    if click_type == 1:
        ten.click()
    if click_type ==2:
        twenty.click()
    if click_type == 3:
        forty.click()
#     click_count += 1

def scrape_table():
    data=driver.find_elements_by_xpath("//table[contains(@id,'result-table')]")[0].text
    splitdata=data.split("\n")
    columns=splitdata[0].split(" ")
    values_split=[]
    for values in splitdata[1:len(splitdata)]:
        values_split.append(values.split(" "))

    table=pd.DataFrame(values_split)
    table.columns=columns[0:4]
    return(table)

# def get_results(results_df, performance_df, game_id):
#     table = driver.find_elements_by_id("result-table")
#     for row in table:
#         for cell_number in range(1, 16):
#             cell = row.find_elements_by_tag_name("tr")[cell_number]
#             print(cell.text)
#             results_list = cell.text.split(' ')
#             print(results_list)
#             results_df = results_df.append({'Round': game_id, 'Week': results_list[0], 'Price': results_list[1], 'Sales': results_list[2],
#                                'Remaining': results_list[3]}, ignore_index=True)

#     revenue = driver.find_element_by_id("rev")
#     perfect = driver.find_element_by_id("perfect")
#     difference = driver.find_element_by_id("percentage")
#     print(f'Your revenue is: {revenue.text}')
#     print(f'Perfect revenue is: {perfect.text}')
#     print(f'Difference is: {difference.text}')
#     performance_df = performance_df.append({'Round': game_id, 'Revenue': revenue.text, 'Perfect': perfect.text, 'Difference': difference.text.split(' ')[2]}, ignore_index=True)

#     return results_df, performance_df

# Try out controls
'''
main_game_click()
main_game_click()
main_game_click()
main_game_click()
main_game_click('ten')
main_game_click()
main_game_click()
main_game_click()
main_game_click('twenty')
main_game_click()
main_game_click()
main_game_click()
main_game_click('forty')
'''



"\nmain_game_click()\nmain_game_click()\nmain_game_click()\nmain_game_click()\nmain_game_click('ten')\nmain_game_click()\nmain_game_click()\nmain_game_click()\nmain_game_click('twenty')\nmain_game_click()\nmain_game_click()\nmain_game_click()\nmain_game_click('forty')\n"

# Tiancheng Strategy

In [8]:
all_data=[]
foresights=[]
driver.find_elements_by_xpath("//input[contains(@id,'practiceButton')]")[0].click()
#choices=[[0]*15,[1]+[0]*14,[2]+[0]*14,[3]+[0]*14] # These are the strategies to scrape
#choices=[choices[0]]
attempts=1000

for attempt in range(attempts):#choices:
    chosenchoices=[0]
    driver.find_elements_by_xpath("//input[contains(@id,'practiceButton')]")[0].click()
    table=scrape_table()
    a=table['Sales'][0]
    model,values=optimizer(int(table['Sales'][0]))
    all_choices=[]
    for i in range(0,len(choices)):
        all_choices.extend([i]*int(values[i]))
    if(len(all_choices)==14):
        all_choices.append(all_choices[len(all_choices)-1])
    if(len(all_choices)<15):
        print("Fixed")
        all_choices.extend([all_choices[len(all_choices)-1]]*(15-(len(all_choices))))
    
    for i in range(1,15):
        main_game_click(click_strategy(all_choices,i))

    foresight=driver.find_element_by_xpath("//div[contains(@style,'position:relative;top:10px;left:6px;')]").text
    table=scrape_table()
    table['foresight']=foresight
    table['Choice']=all_choices
    table['diff']=float(table['foresight'][0].split(": ")[3].replace("%",""))
    table['revenue']=float(table['foresight'][0].split(", ")[0].split(": ")[1].replace("$","").replace(",",""))
    table['perfect']=float(table['foresight'][0].split(", ")[1].split(": ")[1].replace("$","").replace(",",""))
    table['Attempt']=attempt
    all_data.append(table)
# print(table)

Solution Status: Optimal
The result is: $103651.92
Optimal Soultion:
X_0 = 3.0
X_1 = 6.0
X_2 = 6.0
X_3 = 0.0
Solution Status: Optimal
The result is: $110146.68000000001
Optimal Soultion:
X_0 = 7.0
X_1 = 6.0
X_2 = 2.0
X_3 = 0.0
Solution Status: Optimal
The result is: $95359.68
Optimal Soultion:
X_0 = 1.0
X_1 = 0.0
X_2 = 14.0
X_3 = 0.0
Solution Status: Optimal
The result is: $105651.84
Optimal Soultion:
X_0 = 3.0
X_1 = 8.0
X_2 = 4.0
X_3 = 0.0
Solution Status: Optimal
The result is: $72335.76000000001
Optimal Soultion:
X_0 = 1.0
X_1 = 0.0
X_2 = 0.0
X_3 = 14.0
Solution Status: Optimal
The result is: $102440.52
Optimal Soultion:
X_0 = 2.0
X_1 = 6.0
X_2 = 7.0
X_3 = 0.0
Solution Status: Optimal
The result is: $78981.48
Optimal Soultion:
X_0 = 1.0
X_1 = 0.0
X_2 = 5.0
X_3 = 9.0
Solution Status: Optimal
The result is: $101595.6
Optimal Soultion:
X_0 = 4.0
X_1 = 2.0
X_2 = 9.0
X_3 = 0.0
Solution Status: Optimal
The result is: $93051.0
Optimal Soultion:
X_0 = 1.0
X_1 = 0.0
X_2 = 13.0
X_3 = 1.0
Solu

KeyboardInterrupt: 

In [7]:
all_data=pd.concat(all_data)
all_data.to_csv("games_tiancheng.csv")

NameError: name 'all_data' is not defined